# Shortest Path Calculation with PostGIS and pgRouting
This notebook demonstrates how to find and visualize the shortest route between two points using a `roads` table with `id` and `geom` (LineString) columns in PostGIS. Both A* and Dijkstra algorithms are used and compared visually.

# Geospatial Data Processing with PostGIS
Learn how to use PostGIS for spatial data operations, shortest path calculations, and route visualization.

# Metrics and Visualization


In [19]:
# start connection postgis
import psycopg2
conn = psycopg2.connect(dbname="postgres", user="postgres", password="mysecretpassword", host="localhost", port="5435")
cur = conn.cursor()

# Example coordinates for start and end points
start_lon, start_lat = 3644020.287608132, 4859368.559524261
end_lon, end_lat = 3646564.260799282, 4858661.401608404

# Example coordinates for start and end points
start_lon2, start_lat2 = 32.7347912, 39.954748
end_lon2, end_lat2 = 32.7576441, 39.9498783

In [20]:
# Find nearest road segment to start point
# Create roads table from planet_osm_line if it doesn't exist
cur.execute("""
    CREATE TABLE IF NOT EXISTS roads_metric AS 
    SELECT osm_id as id, ST_Transform(way, 3857) as geom 
    FROM planet_osm_line 
    WHERE highway IS NOT NULL;
""")
cur.execute("CREATE INDEX IF NOT EXISTS roads_metric_geom_idx ON roads_metric USING GIST(geom);")
conn.commit()

# Find nearest road segment to start point
cur.execute("""SELECT id FROM roads_metric ORDER BY geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 3857) LIMIT 1;""", (start_lon, start_lat))
start_road_id = cur.fetchone()[0]
# Find nearest road segment to end point
cur.execute("""SELECT id FROM roads_metric ORDER BY geom <-> ST_SetSRID(ST_MakePoint(%s, %s), 3857) LIMIT 1;""", (end_lon, end_lat))
end_road_id = cur.fetchone()[0]
print('Start road id:', start_road_id)
print('End road id:', end_road_id)

conn.commit()

Start road id: 710513863
End road id: 447747710


In [21]:
# Check if edges_custom table exists, if not create it
cur.execute("""
   SELECT EXISTS (
      SELECT FROM information_schema.tables 
      WHERE table_name = 'edges_metric'
   );
""")
edges_exists = cur.fetchone()[0]

if not edges_exists:
   print("--- Creating 'edges_metric' table ---")
   cur.execute("""
   CREATE TABLE edges_metric AS
   SELECT osm_id as id, ST_Transform(way, 3857) AS geom FROM planet_osm_line WHERE highway IS NOT NULL;
   """)
   cur.execute("ALTER TABLE edges_metric ADD PRIMARY KEY (id);")
   cur.execute("ALTER TABLE edges_metric ADD COLUMN source integer;")
   cur.execute("ALTER TABLE edges_metric ADD COLUMN target integer;")
   conn.commit()
   print("✅ 'edges_metric' table created.")
else:
   print("ℹ️ 'edges_metric' table already exists.")

conn.commit()

ℹ️ 'edges_metric' table already exists.


In [22]:
# Check if edges_vertices_pgr table exists, if not create topology
cur.execute("""
   SELECT EXISTS (
      SELECT FROM information_schema.tables 
      WHERE table_name = 'edges_metric_vertices_pgr'
   );
""")
vertices_exists = cur.fetchone()[0]

if not vertices_exists:
   print("--- Creating topology with pgr_createTopology ---")
   cur.execute("""SELECT pgr_createTopology('edges_metric', 2, 'geom', 'id');""")
   conn.commit()
   print("✅ Topology created. Node table: 'edges_metric_vertices_pgr'")
else:
   print("ℹ️ 'edges_metric_vertices_pgr' table already exists.")

conn.commit()

ℹ️ 'edges_metric_vertices_pgr' table already exists.


In [24]:
conn.rollback()

# Start vertex
cur.execute("""
    SELECT id 
    FROM edges_metric_vertices_pgr 
    ORDER BY the_geom <->  ST_SetSRID(ST_MakePoint(%s, %s), 3857)
    LIMIT 1;
""", (start_lon, start_lat))
start_vid = cur.fetchone()[0]

# End vertex
cur.execute("""
    SELECT id 
    FROM edges_metric_vertices_pgr 
    ORDER BY the_geom <->  ST_SetSRID(ST_MakePoint(%s, %s), 3857)
    LIMIT 1;
""", (end_lon, end_lat))
end_vid = cur.fetchone()[0]

print("Start vertex ID:", start_vid)
print("End vertex ID:", end_vid)

cur.execute("""
    SELECT COUNT(*) FROM edges_metric WHERE source IS NULL OR target IS NULL;
""")
null_count = cur.fetchone()[0]
print("Edges with NULL source/target:", null_count)

cur.execute("""
    SELECT COUNT(*) FROM edges_metric_vertices_pgr;
""")
vertex_count = cur.fetchone()[0]
print("Total vertices:", vertex_count)

# Example usage of pgr_aStar for one-to-one shortest path calculation
# Using start_vid and end_vid (vertex IDs) instead of road IDs
query = '''
   SELECT * FROM pgr_aStar(
    'SELECT id, source, target, ST_Length(geom) AS cost, ST_Length(geom) AS reverse_cost,
    ST_X(ST_StartPoint(geom)) AS x1, ST_Y(ST_StartPoint(geom)) AS y1,
    ST_X(ST_EndPoint(geom)) AS x2, ST_Y(ST_EndPoint(geom)) AS y2
    FROM edges_metric',
  %s, %s,
  directed => false, heuristic => 1);
'''
cur.execute(query, (start_vid, end_vid))
result = cur.fetchall()
if result:
    for row in result:
        print(row)
else:
    print("No path found")



Start vertex ID: 96704
End vertex ID: 32101
Edges with NULL source/target: 0
Total vertices: 116108
No path found
No path found


In [25]:
# --- Hata Analizi ve Otomatik Düzeltme ---

# 1. source/target sütunlarında NULL olan kenar sayısını kontrol et
cur.execute("SELECT COUNT(*) FROM edges_metric WHERE source IS NULL OR target IS NULL;")
null_count = cur.fetchone()[0]
print(f"Edges with NULL source/target: {null_count}")

# 2. Eğer çok fazla NULL varsa, tolerance değerini artırarak topolojiyi tekrar oluşturmayı öner
if null_count > 0:
    print("Uyarı: Topolojide NULL source/target var. Tolerance değerini artırarak tekrar oluşturmayı deneyin.")
    print("Örnek: SELECT pgr_createTopology('edges_metric', 10, 'geom', 'id');")

# 3. Başlangıç ve bitiş vertex ID'lerinin yol ağına yakınlığını kontrol et
cur.execute("SELECT id, the_geom FROM edges_metric_vertices_pgr ORDER BY the_geom <-> ST_Transform(ST_SetSRID(ST_MakePoint(%s, %s), 4326), 3857) LIMIT 1;", (start_lon, start_lat))
start_vertex = cur.fetchone()
cur.execute("SELECT id, the_geom FROM edges_metric_vertices_pgr ORDER BY the_geom <-> ST_Transform(ST_SetSRID(ST_MakePoint(%s, %s), 4326), 3857) LIMIT 1;", (end_lon, end_lat))
end_vertex = cur.fetchone()
print(f"Start vertex: {start_vertex[0]}, End vertex: {end_vertex[0]}")

# 4. Yol ağı kopuk mu? Toplam vertex ve edge sayısını kontrol et
cur.execute("SELECT COUNT(*) FROM edges_metric;")
edge_count = cur.fetchone()[0]
cur.execute("SELECT COUNT(*) FROM edges_metric_vertices_pgr;")
vertex_count = cur.fetchone()[0]
print(f"Total edges: {edge_count}, Total vertices: {vertex_count}")

# 5. Yol bulamama nedenleri için öneri
if null_count > 0:
    print("Çözüm: pgr_createTopology fonksiyonunu tolerance değerini artırarak tekrar çalıştırın ve source/target sütunlarının dolduğundan emin olun.")
else:
    print("Başlangıç/bitiş noktalarını yol ağına daha yakın seçmeyi deneyin veya veri bütünlüğünü kontrol edin.")

Edges with NULL source/target: 0


InternalError_: transform: latitude or longitude exceeded limits (-14)


In [10]:
import folium
import json

# Create map centered at the start point
m = folium.Map(location=[start_lat, start_lon], zoom_start=13)

# # --- A* Path ---
# astar_edge_ids = [row[5] for row in result if row[5] != -1]
# if astar_edge_ids:
#     sql = f"SELECT ST_AsGeoJSON(geom) FROM edges_metric WHERE id IN ({','.join(map(str, astar_edge_ids))});"
#     cur.execute(sql)
#     geoms = [row[0] for row in cur.fetchall()]
#     for geojson_str in geoms:
#         coords = json.loads(geojson_str)["coordinates"]
#         folium.PolyLine([(c[1], c[0]) for c in coords], color="red", weight=5, tooltip="A* Path").add_to(m)

# --- Dijkstra Path ---
dijkstra_query = """
SELECT * FROM pgr_dijkstra(
    'SELECT id, source, target, ST_Length(geom) AS cost FROM edges_metric',
    %s, %s,
    directed := false
    );
"""
cur.execute(dijkstra_query, (start_vid, end_vid))
dijkstra_result = cur.fetchall()
dijkstra_edge_ids = [row[5] for row in dijkstra_result if row[5] != -1]
if dijkstra_edge_ids:
    sql = f"SELECT ST_AsGeoJSON(geom) FROM edges_metric WHERE id IN ({','.join(map(str, dijkstra_edge_ids))});"
    cur.execute(sql)
    geoms = [row[0] for row in cur.fetchall()]
    for geojson_str in geoms:
        coords = json.loads(geojson_str)["coordinates"]
        folium.PolyLine([(c[1], c[0]) for c in coords], color="blue", weight=3, tooltip="Dijkstra Path").add_to(m)

# Mark start and end points
folium.Marker([start_lat, start_lon], popup="Start", icon=folium.Icon(color="green")).add_to(m)
folium.Marker([end_lat, end_lon], popup="End", icon=folium.Icon(color="purple")).add_to(m)

# Save map
m.save("shortest_path_comparison_map.html")
print("✅ Both A* and Dijkstra paths visualized: shortest_path_comparison_map.html")

✅ Both A* and Dijkstra paths visualized: shortest_path_comparison_map.html


In [17]:
conn.rollback()

In [36]:
cur.execute("""
SELECT source, target
FROM edges_metric
WHERE source IS NULL OR target IS NULL
LIMIT 5;""")
rows = cur.fetchall()
for row in rows:
    print(row)